In [1]:
from datetime import datetime, timedelta, timezone
import requests
import json

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import Row
from datetime import date
from pyspark.sql.functions import lit, current_timestamp
import pyspark

In [2]:
spark = SparkSession.builder \
    .appName("Clean_PGSoft_Save_to_HDFS") \
    .master("local") \
    .enableHiveSupport() \
    .config("spark.jars", "../postgresql-42.6.0.jar") \
    .getOrCreate()

23/08/11 11:43:17 WARN Utils: Your hostname, tns-des180 resolves to a loopback address: 127.0.1.1; using 172.24.31.35 instead (on interface enp2s0)
23/08/11 11:43:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/11 11:43:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/11 11:43:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
parDF1=spark.read.parquet("./user/hive/datalake/wagers/pgsoft")

In [4]:
# parDF2=spark.read.parquet("./user/hive/datalake/wagers/pgsoft2")

In [5]:
# parDF3=spark.read.parquet("./user/hive/datalake/wagers/pgsoft3")

In [6]:
parDF1.describe().show()

23/08/11 11:43:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+-----------------+-----------------+-----------------+--------+------------------+------------------+----------------------------+-----------------+------------------+------------------+----------+-----------------+-----------------+------------------+-----------------+------------------+
|summary|     balanceAfter|     balanceBefore|        betAmount|            betId|          betType|currency|            gameId|                id|jackpotRtpContributionAmount| jackpotWinAmount|       parentBetId|          platform|playerName|       rowVersion|  transactionType|         winAmount|             year|           quarter|
+-------+-----------------+------------------+-----------------+-----------------+-----------------+--------+------------------+------------------+----------------------------+-----------------+------------------+------------------+----------+-----------------+-----------------+------------------+-----------------+------------

In [7]:
dropDisDF = parDF1.dropDuplicates(["betId"])

In [8]:
dropDisDF.describe().show()

+-------+-----------------+-----------------+------------------+------------------+-----------------+--------+-----------------+------------------+----------------------------+------------------+------------------+------------------+----------+-----------------+-----------------+-----------------+------------------+------------------+
|summary|     balanceAfter|    balanceBefore|         betAmount|             betId|          betType|currency|           gameId|                id|jackpotRtpContributionAmount|  jackpotWinAmount|       parentBetId|          platform|playerName|       rowVersion|  transactionType|        winAmount|              year|           quarter|
+-------+-----------------+-----------------+------------------+------------------+-----------------+--------+-----------------+------------------+----------------------------+------------------+------------------+------------------+----------+-----------------+-----------------+-----------------+------------------+---------

In [9]:
dropDisDF = dropDisDF.na.drop(subset=["betId"]) 

In [10]:
dropDisDF.describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+--------+-----------------+------------------+----------------------------+------------------+------------------+-----------------+----------+-----------------+-----------------+------------------+------------------+------------------+
|summary|     balanceAfter|     balanceBefore|         betAmount|             betId|          betType|currency|           gameId|                id|jackpotRtpContributionAmount|  jackpotWinAmount|       parentBetId|         platform|playerName|       rowVersion|  transactionType|         winAmount|              year|           quarter|
+-------+-----------------+------------------+------------------+------------------+-----------------+--------+-----------------+------------------+----------------------------+------------------+------------------+-----------------+----------+-----------------+-----------------+------------------+------------------+------

In [11]:
# dropDisDF.select("id","name","marks").write.format("jdbc")\
#     .option("url", "jdbc:postgresql://localhost:5432/dezyre_new") \
#     .option("driver", "org.postgresql.Driver").option("dbtable", "students") \
#     .option("user", "hduser").option("password", "bigdata").save()

# Save to Postgres

In [12]:
# import psycopg2

In [13]:
# conn = psycopg2.connect(
#     database="collectorDB",
#     host='172.17.0.1',
#     user='airflow',
#     password='airflow',
#     port='5432'
# )
# cursor = conn.cursor()


In [14]:
dropDisDF.describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+--------+-----------------+------------------+----------------------------+------------------+------------------+-----------------+----------+-----------------+-----------------+------------------+------------------+------------------+
|summary|     balanceAfter|     balanceBefore|         betAmount|             betId|          betType|currency|           gameId|                id|jackpotRtpContributionAmount|  jackpotWinAmount|       parentBetId|         platform|playerName|       rowVersion|  transactionType|         winAmount|              year|           quarter|
+-------+-----------------+------------------+------------------+------------------+-----------------+--------+-----------------+------------------+----------------------------+------------------+------------------+-----------------+----------+-----------------+-----------------+------------------+------------------+------

In [15]:
postgresDF = dropDisDF \
    .withColumnRenamed("balanceAfter","balance_after") \
    .withColumnRenamed("balanceBefore","balance_before") \
    .withColumnRenamed("betAmount","bet_amount") \
    .withColumnRenamed("betId","bet_id") \
    .withColumnRenamed("betType","bet_type") \
    .withColumnRenamed("betTime","bet_time") \
    .withColumnRenamed("gameId","game_id") \
    .withColumnRenamed("gameId","game_id") \
    .withColumnRenamed("jackpotRtpContributionAmount","jackpot_rtp_contribution_amount") \
    .withColumnRenamed("jackpotWinAmount","jackpot_win_amount") \
    .withColumnRenamed("parentBetId","parent_bet_id") \
    .withColumnRenamed("playerName","player_name") \
    .withColumnRenamed("rowVersion","row_version") \
    .withColumnRenamed("transactionType","transaction_type") \
    .withColumnRenamed("winAmount","win_amount") \
    .drop("id") \
    .drop("year") \
    .drop("quarter")

In [16]:
postgresDF = postgresDF.withColumn("update_at", current_timestamp()).withColumn("create_at", current_timestamp())

In [17]:
postgresDF.show()

+-------------+--------------+----------+------+-------------------+--------+--------+-------+-------------------------------+------------------+-------------+--------+-----------+-----------+----------------+----------+--------------------+--------------------+
|balance_after|balance_before|bet_amount|bet_id|           bet_time|bet_type|currency|game_id|jackpot_rtp_contribution_amount|jackpot_win_amount|parent_bet_id|platform|player_name|row_version|transaction_type|win_amount|           update_at|           create_at|
+-------------+--------------+----------+------+-------------------+--------+--------+-------+-------------------------------+------------------+-------------+--------+-----------+-----------+----------------+----------+--------------------+--------------------+
|          857|           795|       198|    87|1979-01-26 13:55:26|     767|   dbkgm|    457|                            315|               285|       809327|     153|      dpfge|        625|             906|  

In [18]:
postgresDF.printSchema()

root
 |-- balance_after: long (nullable = true)
 |-- balance_before: long (nullable = true)
 |-- bet_amount: long (nullable = true)
 |-- bet_id: long (nullable = true)
 |-- bet_time: timestamp (nullable = true)
 |-- bet_type: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- game_id: long (nullable = true)
 |-- jackpot_rtp_contribution_amount: long (nullable = true)
 |-- jackpot_win_amount: long (nullable = true)
 |-- parent_bet_id: long (nullable = true)
 |-- platform: long (nullable = true)
 |-- player_name: string (nullable = true)
 |-- row_version: long (nullable = true)
 |-- transaction_type: long (nullable = true)
 |-- win_amount: long (nullable = true)
 |-- update_at: timestamp (nullable = false)
 |-- create_at: timestamp (nullable = false)



In [19]:
postgresDF.write.format('jdbc').options(
  url='jdbc:postgresql://172.17.0.1:5432/collectorDB',
  driver='org.postgresql.Driver',
  dbtable='old_pgsoft_wager',
  user='postgres',
  password='secret'
  ).mode('Append').save()